Install Dependencies

In [2]:
# Processing libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

#Imports pickle
import pickle

#Library for indicators
from finta import TA

#Another library for indicators
from ta.utils import dropna
from ta.volatility import BollingerBands
from ta.trend import PSARIndicator
from ta.trend import SMAIndicator
from ta.trend import macd
from ta.momentum import rsi
from ta.volume import OnBalanceVolumeIndicator

Selects the dataframe for training

In [3]:
#df = pd.read_csv('/content/drive/MyDrive/Other Projects/USDGBP.csv')
#df = pd.read_csv('/content/drive/MyDrive/Other Projects/USDJPY.csv')
#df = pd.read_csv('/Users/esteban/Documents/GitHub/Reinforcement_Learning_Binary_Options_Bot/BTCUSD_1hr.csv')
df = pd.read_csv('/Users/esteban/Documents/GitHub/Reinforcement_Learning_Binary_Options_Bot/BTCUSD_day.csv')
#df = pd.read_csv('/content/drive/MyDrive/Other Projects/BTCUSD_1hr.csv')
#df = pd.read_csv('/content/drive/MyDrive/Other Projects/BTCUSD_1min.csv')
#df = pd.read_csv('/content/drive/MyDrive/Other Projects/Download Data - STOCK_US_XNYS_F (1).csv')

df

,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USD
0,2020-04-10,BTCUSD,7315.25,7315.25,7315.25,7315.25,0.00,0.00
1,2020-04-09,BTCUSD,7369.60,7378.85,7115.04,7315.25,2237.13,16310014.53
2,2020-04-08,BTCUSD,7201.81,7432.23,7152.80,7369.60,2483.60,18138080.27
3,2020-04-07,BTCUSD,7336.96,7468.42,7078.00,7201.81,2333.34,17047120.32
4,2020-04-06,BTCUSD,6775.21,7369.76,6771.01,7336.96,3727.47,26533750.17
...,...,...,...,...,...,...,...,...
1642,2015-10-12,BTCUSD,249.50,249.50,247.60,247.60,38.28,9493.89
1643,2015-10-11,BTCUSD,246.30,249.50,246.30,249.50,12.22,3021.12
1644,2015-10-10,BTCUSD,245.39,246.30,244.60,246.30,12.17,2984.44
1645,2015-10-09,BTCUSD,243.95,249.97,243.60,245.39,30.99,7651.63


Data Cleanup

In [4]:
print(df.dtypes)

#Convert date
#df['Date'] = pd.to_datetime(df['Date'])

#Sort values from past to present
df.sort_values('Date', ascending=True, inplace=True)

#Set date as index
df.set_index('Date', inplace=True)

#Converts volume from string to float
#df.drop('Volume', inplace=True, axis=1)
#df['Volume'] = df['Volume'].apply(lambda x: float(x.replace(",", "")))
#df['Volume'] = df['Volume'].astype(float)
#df = df.drop(columns=['Symbol'])
#df = df.drop(columns=['Unix Timestamp'])
df = df.drop(columns=['Volume USD'])
df = df.rename(columns={'Volume BTC': 'Volume'})

print(df.shape[0])
print(df.dtypes)

df.head()

Date           object
Symbol         object
Open          float64
High          float64
Low           float64
Close         float64
Volume BTC    float64
Volume USD    float64
dtype: object
1647
Symbol     object
Open      float64
High      float64
Low       float64
Close     float64
Volume    float64
dtype: object


,Symbol,Open,High,Low,Close,Volume
Date,,,,,,
2015-10-08,BTCUSD,242.50,245.00,242.5,243.95,18.80
2015-10-09,BTCUSD,243.95,249.97,243.6,245.39,30.99
2015-10-10,BTCUSD,245.39,246.30,244.6,246.30,12.17
2015-10-11,BTCUSD,246.30,249.50,246.3,249.50,12.22
2015-10-12,BTCUSD,249.50,249.50,247.6,247.60,38.28


Normalization

In [ ]:
'''
df["Open"] = df["Open"] - df["Open"].shift(1)
df["High"] = df["High"] - df["High"].shift(1)
df["Low"] = df["Low"] - df["Low"].shift(1)
df["Close"] = df["Close"] - df["Close"].shift(1)
'''

'\ndf["Open"] = df["Open"] - df["Open"].shift(1)\ndf["High"] = df["High"] - df["High"].shift(1)\ndf["Low"] = df["Low"] - df["Low"].shift(1)\ndf["Close"] = df["Close"] - df["Close"].shift(1)\n'

# 2.1 Add indicators
2.1.1 Install New Dependencies

2.1.3. Calculate SMA, RSI and OBV


In [5]:

#Indicators with TA
#Adds simple moving average
df["sma7"] = SMAIndicator(close=df["Open"], window=7, fillna=True).sma_indicator()
df["sma25"] = SMAIndicator(close=df["Open"], window=25, fillna=True).sma_indicator()
df["sma99"] = SMAIndicator(close=df["Open"], window=99, fillna=True).sma_indicator()

#Adds Bollinger Bands
indicator_bb = BollingerBands(close=df["Open"], window=20, window_dev=2)
df['bb_bbm'] = indicator_bb.bollinger_mavg()
df['bb_bbh'] = indicator_bb.bollinger_hband()
df['bb_bbl'] = indicator_bb.bollinger_lband()

#Adds PSAR Indicator
indicator_psar = PSARIndicator(high=df["Close"], low=df["Low"], close=df["Close"], step=0.02, max_step=2, fillna=True)
df['psar'] = indicator_psar.psar()

# Add Moving Average Convergence Divergence (MACD) indicator
df["MACD"] = macd(close=df["Close"], window_slow=26, window_fast=12, fillna=True)
    
# Add Relative Strength Index (RSI) indicator
df["RSI"] = rsi(close=df["Close"], window=14, fillna=True)

#Indicators using finta
#Adds simple moving average
df["sma40"] = TA.SMA(df, period = 40)

# Add Relative Strength Index (RSI) indicator
df["OBV"] = TA.OBV(df)

#Drops empty spaces
df.fillna(0, inplace=True)

#Drops empty spaces
df.head(50)


/Users/esteban/miniforge3/lib/python3.9/site-packages/ta/trend.py:938: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_up = pd.Series(index=self._psar.index)
/Users/esteban/miniforge3/lib/python3.9/site-packages/ta/trend.py:939: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_down = pd.Series(index=self._psar.index)


,Symbol,Open,High,Low,Close,Volume,sma7,sma25,sma99,bb_bbm,bb_bbh,bb_bbl,psar,MACD,RSI,sma40,OBV
Date,,,,,,,,,,,,,,,,,
2015-10-08,BTCUSD,242.50,245.00,242.50,243.95,18.80,242.500000,242.500000,242.500000,0.0000,0.000000,0.000000,243.950000,0.000000,100.000000,0.00000,0.00
2015-10-09,BTCUSD,243.95,249.97,243.60,245.39,30.99,243.225000,243.225000,243.225000,0.0000,0.000000,0.000000,245.390000,0.114872,100.000000,0.00000,30.99
2015-10-10,BTCUSD,245.39,246.30,244.60,246.30,12.17,243.946667,243.946667,243.946667,0.0000,0.000000,0.000000,243.950000,0.276155,100.000000,0.00000,43.16
2015-10-11,BTCUSD,246.30,249.50,246.30,249.50,12.22,244.535000,244.535000,244.535000,0.0000,0.000000,0.000000,244.600000,0.654639,100.000000,0.00000,55.38
2015-10-12,BTCUSD,249.50,249.50,247.60,247.60,38.28,245.528000,245.528000,245.528000,0.0000,0.000000,0.000000,244.600000,0.792146,72.095821,0.00000,17.10
2015-10-13,BTCUSD,247.60,252.87,245.75,250.00,51.71,245.873333,245.873333,245.873333,0.0000,0.000000,0.000000,244.698000,1.082304,79.773530,0.00000,68.81
2015-10-14,BTCUSD,250.00,254.75,250.00,253.00,119.54,246.462857,246.462857,246.462857,0.0000,0.000000,0.000000,244.910080,1.536619,85.240336,0.00000,188.35
2015-10-15,BTCUSD,253.00,258.98,253.00,255.26,197.11,247.962857,247.280000,247.280000,0.0000,0.000000,0.000000,245.395475,2.055337,87.894702,0.00000,385.46
2015-10-16,BTCUSD,255.90,269.98,255.90,263.75,101.13,249.670000,248.237778,248.237778,0.0000,0.000000,0.000000,246.184637,3.115582,92.992831,0.00000,486.59


In [10]:
#add a column to dataframe

df['Position'] = np.where((df['Open']-df['Close'].shift(-1)) < 1, True, False)

#df["Position"] = df['Open'] - df['Close'].shift(-1)



In [11]:
df

,Symbol,Open,High,Low,Close,Volume,sma7,sma25,sma99,bb_bbm,bb_bbh,bb_bbl,psar,MACD,RSI,sma40,OBV,Position
Date,,,,,,,,,,,,,,,,,,
2015-10-08,BTCUSD,242.50,245.00,242.50,243.95,18.80,242.500000,242.500000,242.500000,0.0000,0.000000,0.000000,243.950000,0.000000,100.000000,0.00000,0.00,True
2015-10-09,BTCUSD,243.95,249.97,243.60,245.39,30.99,243.225000,243.225000,243.225000,0.0000,0.000000,0.000000,245.390000,0.114872,100.000000,0.00000,30.99,True
2015-10-10,BTCUSD,245.39,246.30,244.60,246.30,12.17,243.946667,243.946667,243.946667,0.0000,0.000000,0.000000,243.950000,0.276155,100.000000,0.00000,43.16,True
2015-10-11,BTCUSD,246.30,249.50,246.30,249.50,12.22,244.535000,244.535000,244.535000,0.0000,0.000000,0.000000,244.600000,0.654639,100.000000,0.00000,55.38,True
2015-10-12,BTCUSD,249.50,249.50,247.60,247.60,38.28,245.528000,245.528000,245.528000,0.0000,0.000000,0.000000,244.600000,0.792146,72.095821,0.00000,17.10,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-06,BTCUSD,6775.21,7369.76,6771.01,7336.96,3727.47,6669.842857,6125.836000,8160.088182,6355.7180,7229.164087,5482.271913,5330.828725,-77.430673,57.402004,7009.73250,402187.43,True
2020-04-07,BTCUSD,7336.96,7468.42,7078.00,7201.81,2333.34,6802.707143,6226.096400,8159.548182,6455.4125,7298.400087,5612.424913,5531.441853,-32.150404,55.331042,6969.53125,399854.09,True
2020-04-08,BTCUSD,7201.81,7432.23,7152.80,7369.60,2483.60,6913.174286,6288.047600,8159.402121,6544.4805,7303.485116,5785.475884,5711.993667,17.076934,57.386583,6936.14925,402337.69,True


*2.1*.4. Dump dataset into a pickle file

In [12]:
import pickle
with open("BTCUSD_1day.pickle", 'wb') as f:
      pickle.dump(df,f, pickle.HIGHEST_PROTOCOL)
